In [56]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from tqdm import tqdm

In [65]:
links=[]


base_site = "https://www.goodreads.com/book/popular_by_date/2023/2"

# Make http request
response = requests.get(base_site)

# Get the html from webpage
html = response.content

# Creating a BeautifulSoup object with the use of a parser
soup = BeautifulSoup(html, "lxml")

divs = soup.find_all("div",{"class":"BookCard"})



In [67]:
for div in divs :
    soup = BeautifulSoup(str(div), 'html.parser')
    url = soup.a['href']
    links.append(url)

In [68]:
links

['https://www.goodreads.com/book/show/61109599-secretly-yours',
 'https://www.goodreads.com/book/show/62022434-things-we-hide-from-the-light',
 'https://www.goodreads.com/book/show/60766189-a-day-of-fallen-night',
 'https://www.goodreads.com/book/show/61053829-i-have-some-questions-for-you',
 'https://www.goodreads.com/book/show/61273371-looking-for-jane',
 'https://www.goodreads.com/book/show/61150759-the-last-tale-of-the-flower-bride',
 'https://www.goodreads.com/book/show/60784841-she-is-a-haunting',
 'https://www.goodreads.com/book/show/60478017-radiant-sin',
 'https://www.goodreads.com/book/show/62848145-the-housemaid-s-secret',
 'https://www.goodreads.com/book/show/62971668-someone-else-s-shoes',
 'https://www.goodreads.com/book/show/61102615-stone-blind',
 'https://www.goodreads.com/book/show/60784591-a-mystery-of-mysteries',
 'https://www.goodreads.com/book/show/61111246-victory-city',
 'https://www.goodreads.com/book/show/60877589-venco',
 'https://www.goodreads.com/book/show/

In [87]:
full_url = [urljoin(base_site, relativeurl) for relativeurl in links]
book_url = [url for url in full_url if "https://www.goodreads.com/book/show" in url]

In [88]:
book_description = []
book_author = []
book_title = []
book_rating = []
book_pages = []
num_of_rating = []
num_of_review = []
genre = []
publish = []
language = []
award = []
isbn = []

In [89]:
from tqdm import tqdm

In [90]:
import re
import json

In [91]:
faulty=[]

In [92]:
for url in tqdm(book_url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    note_resp = requests.get(url, headers=headers)

    #if note_resp.status_code == 200:
     #   print("URL{}: {}".format(i+1, url))
    
    #else:
     #   print('Status code{}: Skipping URL #{}: {}'.format(note_resp.status_code, i+1, url))
      #  i = i+1
       # continue
    
    try:
        #get HTML from url page
        note_html = note_resp.content

        #create beautifulsoup object for url page
        note_soup = BeautifulSoup(note_html, 'html.parser')

        #Extract Author particulars
        author_divs = note_soup.find_all("span",{"class":"ContributorLink__name"})
        author_text = author_divs[0].text
        book_author.append(author_text)

        description_divs = note_soup.find_all("span", {"class": "Formatted"})
        try:
            description_text = description_divs[0].text
        except IndexError:
            description_text = "Nil"
        book_description.append(description_text)

        title_divs = note_soup.find_all("h1", {"class": "Text Text__title1"})
        title_text = title_divs[0].text
        book_title.append(title_text)

        rating_divs = note_soup.find_all("div", {"class": "RatingStatistics__rating"})
        rating_text = rating_divs[0].text
        book_rating.append(float(rating_text))

        pages_divs = note_soup.find_all("p", {"data-testid": "pagesFormat"})
        pages_text = pages_divs[0].text
        book_pages.append(float(re.search(r'\d+', pages_text.replace(',', '')).group()))

        num_of_rating_divs = note_soup.find_all("span", {"data-testid": "ratingsCount"})
        num_of_rating_text = num_of_rating_divs[0].text
        num_of_rating.append(float(re.search(r'\d+', num_of_rating_text.replace(',', '')).group()))

        num_of_review_divs = note_soup.find_all("span", {"data-testid": "reviewsCount"})
        num_of_review_text = num_of_review_divs[0].text
        num_of_review.append(float(re.search(r'\d+', num_of_review_text.replace(',', '')).group()))

        genre_divs = note_soup.find_all("a", {"class": "Button Button--tag-inline Button--small"})
        genre_text = genre_divs[1].text
        genre.append(genre_text)

        publish_divs = note_soup.find_all("p", {"data-testid":"publicationInfo"})
        publish_text = publish_divs[0].text
        publish.append(publish_text.replace("First published ", ""))

        json_str = note_soup.find_all("script",{'type':'application/ld+json'})[0].string
        data = json.loads(json_str)
        language.append(data.get("inLanguage"))

        if 'awards' in data.keys():
            award.append(1)
        else :
            award.append(0)

        if 'isbn' in data.keys():
            isbn.append(data.get("isbn"))
        else :
            isbn.append('NA')
    except :
        faulty.append(url)

100%|███████████████████████████████████████████| 15/15 [00:24<00:00,  1.62s/it]


In [94]:
#adjust list size if any erraneous value enters

book_description = book_description[:len(genre)]
book_author = book_author[:len(genre)]
book_title = book_title[:len(genre)]
book_rating = book_rating[:len(genre)]
book_pages = book_pages[:len(genre)]
num_of_rating = num_of_rating[:len(genre)]
num_of_review = num_of_review[:len(genre)]
num_of_review = num_of_review[:len(genre)]
genre = genre[:len(genre)]
publish = publish[:len(genre)]
language = language[:len(genre)]
award = award[:len(genre)]
isbn = isbn[:len(genre)]

In [95]:
final=pd.DataFrame({'Description':book_description,
'Author':book_author,
'Title':book_title,
'Rating':book_rating,
'Pages':book_pages,
'Num of Ratings':num_of_rating,
'Num of Review':num_of_review,
'Genre':genre,
'Publish Date':publish,
'Language':language,
'Award':award,
'ISBN':isbn})

In [96]:
final

,Description,Author,Title,Rating,Pages,Num of Ratings,Num of Review,Genre,Publish Date,Language,Award,ISBN
0,A steamy new rom-com about a starchy professor...,Tessa Bailey,Secretly Yours,3.82,384.0,1114.0,635.0,Contemporary,"Expected publication February 7, 2023",English,0,9780063238985
1,Nash Morgan was always known as the good Morga...,Lucy Score,Things We Hide From The Light,4.29,880.0,336.0,290.0,Adult,"Expected publication February 28, 2023",English,0,9781635577921
2,"The stunning, standalone prequel to the New Yo...",Samantha Shannon,A Day of Fallen Night,4.70,400.0,335.0,139.0,Fiction,"Expected publication February 21, 2023",English,0,9780593490143
3,"Named a Most Anticipated Book of 2023 by TIME,...",Rebecca Makkai,I Have Some Questions for You,4.11,400.0,7080.0,1053.0,Fiction,"March 1, 2022",English,0,9781668013687
4,A debut about three women whose lives are boun...,Heather Marshall,Looking for Jane,4.48,304.0,872.0,526.0,Adult,"Expected publication February 14, 2023",None,0,9780063206502
5,"A sumptuous, gothic-infused story about a marr...",Roshani Chokshi,The Last Tale of the Flower Bride,4.04,352.0,185.0,90.0,Young Adult,"Expected publication February 28, 2023",English,0,9781547610815
6,A house with a terrifying appetite haunts a br...,Trang Thanh Tran,She Is a Haunting,4.01,320.0,496.0,289.0,Fantasy,"Expected publication February 7, 2023",English,0,9781728257006
7,There's nowhere more dangerous than Olympus......,Katee Robert,Radiant Sin,4.07,227.0,823.0,440.0,Mystery,"Expected publication February 20, 2023",None,0,NA
8,“Don’t go in the guest bedroom.” A shadow fall...,Freida McFadden,The Housemaid's Secret,4.56,448.0,273.0,116.0,Romance,"February 2, 2023",English,0,9781984879295
9,"A story of mix-ups, mess-ups and making the mo...",Jojo Moyes,Someone Else's Shoes,4.16,384.0,5036.0,935.0,Fantasy,"September 15, 2022",English,0,9780063258396
